In [1]:
import pulp as p
import pandas as pd
import math
solver_list = p.listSolvers(onlyAvailable=True)
solver_list

['PULP_CBC_CMD']

In [2]:
# Example data
origin_df = pd.read_excel('Input excel files/Origins.xlsx')
origin_df.columns = ['ID', 'Latitude', 'Longitude', 'M1', 'M2', 'M3']
destinations_df = pd.read_excel('Input excel files/Destinations.xlsx')

origins = origin_df['ID']
destinations = destinations_df['ID']

materials = ['M1', 'M2', 'M3']

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of Earth in kilometers
    R = 6371.0
    distance = R * c

    return distance

# Example usage:
lat1, lon1 = 52.5200, 13.4050  # Berlin
lat2, lon2 = 48.8566, 2.3522   # Paris

distance = haversine(lat1, lon1, lat2, lon2)
print(f"The Haversine distance is {distance:.2f} km.")

The Haversine distance is 877.46 km.


In [4]:
# import openrouteservice
# from openrouteservice import convert

# # Your OpenRouteService API key
# api_key = '5b3ce3597851110001cf62488cd8e293f41043f5a4d5abc7dc952346'
# client = openrouteservice.Client(key=api_key)

In [5]:
# coords = ((78.296757,18.387885),(80.900000,16.767222))
# routes = client.directions(coords, profile='foot-walking')

In [6]:
#routes['routes'][0]['summary']['distance']

In [7]:
# Function to get road distance between two points
# def get_road_distance(coords):
#     routes = client.directions(coords)
#     distance = routes['routes'][0]['summary']['distance']
#     return distance
    


# transport_costs = {
#     ('O1', 'D1'): 2,
#     ('O1', 'D2'): 4,
#     ('O2', 'D1'): 3,
#     ('O2', 'D2'): 1
# }

# This can be calculated for each O-D pair

In [8]:
# import time
# # Calculate road distances
# distances = []
# coordss = []
# routes = []

# for idx in range(2):
#     start = (origin_df.loc[idx, 'Longitude'], origin_df.loc[idx, 'Latitude'])
#     for idx2 in range(2):
#         end = (destinations_df.loc[idx2, 'Longitude'], destinations_df.loc[idx2, 'Latitude'])
#         coords = (start, end)
#         coordss.append(coords)
#         try:
#             distance = get_road_distance(coords)
#             distances.append(distance)
#             routes.append((origin_df.loc[idx, 'ID'], destinations_df.loc[idx2, 'ID']))
#             time.sleep(20)
#         except:
#             print(origin_df.loc[idx, 'ID'])
#             print(destinations_df.loc[idx2, 'ID'])

In [7]:
# Haversine distances
distances = []
routes = []
for idx in range(len(origin_df)):
    lon1 = origin_df.loc[idx, 'Longitude']
    lat1 = origin_df.loc[idx, 'Latitude']
    for idx2 in range(len(destinations_df)):
        lon2 = destinations_df.loc[idx2, 'Longitude']
        lat2 = destinations_df.loc[idx2, 'Latitude']
        try:
            distance = haversine(lat1, lon1, lat2, lon2)
            distances.append(distance)
            routes.append((origin_df.loc[idx, 'ID'], destinations_df.loc[idx2, 'ID']))
        except:
            print(origin_df.loc[idx, 'ID'])
            print(destinations_df.loc[idx2, 'ID'])

In [8]:
distance_matrix = pd.DataFrame([routes,distances]).T
distance_matrix.columns = ['route','km']
# transport_costs = {
#     ('O1', 'D1'): 2,
#     ('O1', 'D2'): 4,
#     ('O2', 'D1'): 3,
#     ('O2', 'D2'): 1
# }

distance_km = distance_matrix.set_index('route').to_dict()['km']

In [9]:
distance_matrix.head()

,route,km
0,"(S1, D1)",249.079252
1,"(S1, D2)",329.5711
2,"(S1, D3)",147.059368
3,"(S1, D4)",148.388496
4,"(S1, D5)",239.368742


In [11]:
supply = origin_df[['ID', 'M1', 'M2', 'M3']].set_index('ID').T.to_dict()
supply

{'S1': {'M1': 138247.4, 'M2': 4200.0, 'M3': 0.0},
 'S2': {'M1': 130814.79999999999, 'M2': 4200.0, 'M3': 0.0},
 'S3': {'M1': 62542.0, 'M2': 4200.0, 'M3': 0.0},
 'S4': {'M1': 66116.0, 'M2': 4200.0, 'M3': 0.0},
 'S5': {'M1': 23056.0, 'M2': 840.0, 'M3': 0.0},
 'S6': {'M1': 16745.4, 'M2': 5880.0, 'M3': 0.0},
 'S7': {'M1': 57478.6, 'M2': 0.0, 'M3': 2503900.0},
 'S8': {'M1': 135758.00000000003, 'M2': 0.0, 'M3': 1128000.0},
 'S9': {'M1': 16570.0, 'M2': 0.0, 'M3': 3875000.0},
 'S10': {'M1': 67635.59999999999, 'M2': 0.0, 'M3': 1571300.0},
 'S11': {'M1': 123369.59999999999, 'M2': 0.0, 'M3': 89000.0}}

In [12]:
demand_total = dict(zip(destinations_df['ID'], destinations_df['Required quantity']))
demand_total

{'D1': 438000,
 'D2': 426000,
 'D3': 80000,
 'D4': 200000,
 'D5': 460000,
 'D6': 59400,
 'D7': 116800,
 'D8': 2200,
 'D9': 1680000,
 'D10': 520000,
 'D11': 600000,
 'D12': 240000,
 'D13': 400000,
 'D14': 554000,
 'D15': 600000,
 'D16': 500000,
 'D17': 480000,
 'D18': 260000,
 'D19': 520000}

In [13]:
# material_costs = {
#     'M1': 5,
#     'M2': 3,
#     'M3': 1
# }

In [14]:
# Calculate demands for M1, M2, M3 at each destination
# demand = {}
# for d in destinations:
#     total = demand_total[d]
#     demand[d] = {'M1': 3 * (total / (3 + 4 + 4)), 'M2': 4 * (total / (3 + 4 + 4)), 'M3': 4 * (total / (3 + 4 + 4))}


In [15]:
# Create a LP Minimization problem 
Lp_prob = p.LpProblem('Problem', sense = p.LpMinimize)  

In [16]:
# Decision variables
x = p.LpVariable.dicts("transport", 
                          [(o, d, m) for o in origins for d in destinations for m in materials], 
                          lowBound=0,
                          cat='Continuous')

x

{('S1', 'D1', 'M1'): transport_('S1',_'D1',_'M1'),
 ('S1', 'D1', 'M2'): transport_('S1',_'D1',_'M2'),
 ('S1', 'D1', 'M3'): transport_('S1',_'D1',_'M3'),
 ('S1', 'D2', 'M1'): transport_('S1',_'D2',_'M1'),
 ('S1', 'D2', 'M2'): transport_('S1',_'D2',_'M2'),
 ('S1', 'D2', 'M3'): transport_('S1',_'D2',_'M3'),
 ('S1', 'D3', 'M1'): transport_('S1',_'D3',_'M1'),
 ('S1', 'D3', 'M2'): transport_('S1',_'D3',_'M2'),
 ('S1', 'D3', 'M3'): transport_('S1',_'D3',_'M3'),
 ('S1', 'D4', 'M1'): transport_('S1',_'D4',_'M1'),
 ('S1', 'D4', 'M2'): transport_('S1',_'D4',_'M2'),
 ('S1', 'D4', 'M3'): transport_('S1',_'D4',_'M3'),
 ('S1', 'D5', 'M1'): transport_('S1',_'D5',_'M1'),
 ('S1', 'D5', 'M2'): transport_('S1',_'D5',_'M2'),
 ('S1', 'D5', 'M3'): transport_('S1',_'D5',_'M3'),
 ('S1', 'D6', 'M1'): transport_('S1',_'D6',_'M1'),
 ('S1', 'D6', 'M2'): transport_('S1',_'D6',_'M2'),
 ('S1', 'D6', 'M3'): transport_('S1',_'D6',_'M3'),
 ('S1', 'D7', 'M1'): transport_('S1',_'D7',_'M1'),
 ('S1', 'D7', 'M2'): transport_

In [17]:
# Binary variables to select blending option
y = p.LpVariable.dicts("y", 
                    [(d, option) for d in destinations for option in ['M2', 'M3']], 
                          cat='Binary')
y

{('D1', 'M2'): y_('D1',_'M2'),
 ('D1', 'M3'): y_('D1',_'M3'),
 ('D2', 'M2'): y_('D2',_'M2'),
 ('D2', 'M3'): y_('D2',_'M3'),
 ('D3', 'M2'): y_('D3',_'M2'),
 ('D3', 'M3'): y_('D3',_'M3'),
 ('D4', 'M2'): y_('D4',_'M2'),
 ('D4', 'M3'): y_('D4',_'M3'),
 ('D5', 'M2'): y_('D5',_'M2'),
 ('D5', 'M3'): y_('D5',_'M3'),
 ('D6', 'M2'): y_('D6',_'M2'),
 ('D6', 'M3'): y_('D6',_'M3'),
 ('D7', 'M2'): y_('D7',_'M2'),
 ('D7', 'M3'): y_('D7',_'M3'),
 ('D8', 'M2'): y_('D8',_'M2'),
 ('D8', 'M3'): y_('D8',_'M3'),
 ('D9', 'M2'): y_('D9',_'M2'),
 ('D9', 'M3'): y_('D9',_'M3'),
 ('D10', 'M2'): y_('D10',_'M2'),
 ('D10', 'M3'): y_('D10',_'M3'),
 ('D11', 'M2'): y_('D11',_'M2'),
 ('D11', 'M3'): y_('D11',_'M3'),
 ('D12', 'M2'): y_('D12',_'M2'),
 ('D12', 'M3'): y_('D12',_'M3'),
 ('D13', 'M2'): y_('D13',_'M2'),
 ('D13', 'M3'): y_('D13',_'M3'),
 ('D14', 'M2'): y_('D14',_'M2'),
 ('D14', 'M3'): y_('D14',_'M3'),
 ('D15', 'M2'): y_('D15',_'M2'),
 ('D15', 'M3'): y_('D15',_'M3'),
 ('D16', 'M2'): y_('D16',_'M2'),
 ('D16', 'M3'

In [18]:
# Objective function
pertonkm_cost = 8.24

# Objective function: Minimize transportation cost
Lp_prob += p.lpSum([pertonkm_cost * distance_km[(o, d)] * x[o, d, m] for o in origins for d in destinations for m in materials])
Lp_prob

Problem:
MINIMIZE
2052.4130383914526*transport_('S1',_'D1',_'M1') + 2052.4130383914526*transport_('S1',_'D1',_'M2') + 2052.4130383914526*transport_('S1',_'D1',_'M3') + 2028.180413150882*transport_('S1',_'D10',_'M1') + 2028.180413150882*transport_('S1',_'D10',_'M2') + 2028.180413150882*transport_('S1',_'D10',_'M3') + 1087.7290574610004*transport_('S1',_'D11',_'M1') + 1087.7290574610004*transport_('S1',_'D11',_'M2') + 1087.7290574610004*transport_('S1',_'D11',_'M3') + 1958.1755728212484*transport_('S1',_'D12',_'M1') + 1958.1755728212484*transport_('S1',_'D12',_'M2') + 1958.1755728212484*transport_('S1',_'D12',_'M3') + 1215.4577857943218*transport_('S1',_'D13',_'M1') + 1215.4577857943218*transport_('S1',_'D13',_'M2') + 1215.4577857943218*transport_('S1',_'D13',_'M3') + 2045.481909930016*transport_('S1',_'D14',_'M1') + 2045.481909930016*transport_('S1',_'D14',_'M2') + 2045.481909930016*transport_('S1',_'D14',_'M3') + 2018.5543505712174*transport_('S1',_'D15',_'M1') + 2018.5543505712174*tra

In [19]:
# Constraints
# 1. Supply constraints
for o in origins:
    for m in materials:
        Lp_prob += p.lpSum([x[o, d, m] for d in destinations]) <= supply[o][m], f"Supply_Constraint_{o}_{m}"

In [20]:
# 2. Demand constraints with OR condition
for d in destinations:
    Lp_prob += p.lpSum([x[o, d, 'M1'] for o in origins]) == 0.05 * demand_total[d], f"Demand_Constraint_M1_{d}"
    Lp_prob += p.lpSum([x[o, d, 'M2'] for o in origins]) == 0.10 * demand_total[d] * y[d, 'M2'], f"Demand_Constraint_M2_{d}"
    Lp_prob += p.lpSum([x[o, d, 'M3'] for o in origins]) == 0.20 * demand_total[d] * y[d, 'M3'], f"Demand_Constraint_M3_{d}"
    Lp_prob += y[d, 'M2'] + y[d, 'M3'] == 1, f"Blending_Option_Selection_{d}"

In [21]:
# 3. Blending constraints
# for d in destinations:
#     Lp_prob += p.lpSum([x[o, d, 'M2'] for o in origins]) == 2 * p.lpSum([x[o, d, 'M1'] for i in origins]), f"Blending_Constraint_M2_{d}"
#     Lp_prob += p.lpSum([x[o, d, 'M3'] for o in origins]) == 4 * p.lpSum([x[o, d, 'M1'] for i in origins]), f"Blending_Constraint_M3_{d}"

In [22]:
# Solve the problem
Lp_prob.solve()

1

In [23]:
print("Total Cost = ", p.value(Lp_prob.objective))

Total Cost =  2062705286.945659


In [24]:
print(f"Status: {p.LpStatus[Lp_prob.status]}")

Status: Optimal


# Results

In [25]:
distance_matrix[['SupplyPoint', 'DemandPoint']] = pd.DataFrame(distance_matrix['route'].tolist(), index=distance_matrix.index)

In [26]:
import regex as re

loads = []
routes = []
for v in Lp_prob.variables():
    if 'trans' in v.name:
        routes.append(re.findall(r"transport_\('([^']*)',_'([^']*)',_'([^']*)'\)", v.name)[0])
        loads.append(v.varValue)

loads_matrix = pd.DataFrame([routes,loads]).T
loads_matrix.columns = ['route','load']

loads_matrix[['SupplyPoint', 'DemandPoint', 'Material']] = pd.DataFrame(loads_matrix['route'].tolist(), index=loads_matrix.index)
loads_matrix

,route,load,SupplyPoint,DemandPoint,Material
0,"(S1, D1, M1)",0.0,S1,D1,M1
1,"(S1, D1, M2)",0.0,S1,D1,M2
2,"(S1, D1, M3)",0.0,S1,D1,M3
3,"(S1, D10, M1)",0.0,S1,D10,M1
4,"(S1, D10, M2)",0.0,S1,D10,M2
...,...,...,...,...,...
622,"(S9, D8, M2)",0.0,S9,D8,M2
623,"(S9, D8, M3)",0.0,S9,D8,M3
624,"(S9, D9, M1)",0.0,S9,D9,M1
625,"(S9, D9, M2)",0.0,S9,D9,M2


In [27]:
finalresults = loads_matrix.merge(distance_matrix, on=['SupplyPoint','DemandPoint'])
finalresults['Cost'] = finalresults['load']*finalresults['km']*pertonkm_cost

In [28]:
finalresults = finalresults[['SupplyPoint', 'DemandPoint', 'Material', 'load', 'km', 'Cost']]
finalresults

,SupplyPoint,DemandPoint,Material,load,km,Cost
0,S1,D1,M1,0.0,249.079252,0.0
1,S1,D1,M2,0.0,249.079252,0.0
2,S1,D1,M3,0.0,249.079252,0.0
3,S1,D10,M1,0.0,246.1384,0.0
4,S1,D10,M2,0.0,246.1384,0.0
...,...,...,...,...,...,...
622,S9,D8,M2,0.0,210.901719,0.0
623,S9,D8,M3,0.0,210.901719,0.0
624,S9,D9,M1,0.0,221.669252,0.0
625,S9,D9,M2,0.0,221.669252,0.0


In [29]:
from natsort import natsort_keygen

finalresults[finalresults.load != 0].sort_values(['DemandPoint','SupplyPoint'],key=natsort_keygen()).to_excel('final_results.xlsx', index=False)

In [47]:
supplied = finalresults.groupby(['SupplyPoint','Material'])[['load']].sum().reset_index()
supplied = supplied.pivot_table(index='SupplyPoint',columns='Material',values='load').reset_index()
supplied

Material,SupplyPoint,M1,M2,M3
0,S1,0.0,4200.0,0.0
1,S10,30000.0,0.0,120000.0
2,S11,4000.0,0.0,89000.0
3,S2,130814.8,900.0,0.0
4,S3,62542.0,4200.0,0.0
5,S4,20000.0,4200.0,0.0
6,S5,17254.6,840.0,0.0
7,S6,16745.4,5880.0,0.0
8,S7,57478.6,0.0,1274840.0
9,S8,67984.6,0.0,0.0


In [48]:
merged_df = pd.merge(supplied, origin_df, left_on='SupplyPoint', right_on='ID', suffixes=('_df1', '_df2'))
merged_df

,SupplyPoint,M1_df1,M2_df1,M3_df1,ID,Latitude,Longitude,M1_df2,M2_df2,M3_df2
0,S1,0.0,4200.0,0.0,S1,18.387885,78.296757,138247.4,4200,0
1,S10,30000.0,0.0,120000.0,S10,18.837061,79.574244,67635.6,0,1571300
2,S11,4000.0,0.0,89000.0,S11,18.754928,79.456186,123369.6,0,89000
3,S2,130814.8,900.0,0.0,S2,17.187768,80.565288,130814.8,4200,0
4,S3,62542.0,4200.0,0.0,S3,17.131944,80.007500,62542.0,4200,0
5,S4,20000.0,4200.0,0.0,S4,18.011909,78.248737,66116.0,4200,0
6,S5,17254.6,840.0,0.0,S5,18.235278,77.913056,23056.0,840,0
7,S6,16745.4,5880.0,0.0,S6,17.732504,77.596885,16745.4,5880,0
8,S7,57478.6,0.0,1274840.0,S7,17.610286,80.698142,57478.6,0,2503900
9,S8,67984.6,0.0,0.0,S8,18.366400,79.872417,135758.0,0,1128000


In [81]:
# List of columns to compare
metrics = ['M1', 'M2', 'M3']

# Calculate differences
for metric in metrics:
    merged_df[f'{metric}_diff'] = merged_df[f'{metric}_df2'] - merged_df[f'{metric}_df1']

# Drop the original columns --  only want the differences
result_df = merged_df[['ID'] + [f'{metric}_diff' for metric in metrics]]
result_df['M1_diff'] = result_df['M1_diff'].abs()
result_df['M2_diff'] = result_df['M2_diff'].abs()
result_df['M3_diff'] = result_df['M3_diff'].abs()

C:\Users\dskcy\AppData\Local\Temp\ipykernel_22956\1929670317.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['M1_diff'] = result_df['M1_diff'].abs()
C:\Users\dskcy\AppData\Local\Temp\ipykernel_22956\1929670317.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['M2_diff'] = result_df['M2_diff'].abs()
C:\Users\dskcy\AppData\Local\Temp\ipykernel_22956\1929670317.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [82]:
result_df.sort_values(['ID'],key=natsort_keygen()).to_excel('leftover_surplus.xlsx', index=False)

In [58]:
demands = pd.DataFrame.from_dict(demand_total, orient='index').reset_index()
demands.columns = ['DemandPoint','Demand']
demands

,DemandPoint,Demand
0,D1,438000
1,D2,426000
2,D3,80000
3,D4,200000
4,D5,460000
5,D6,59400
6,D7,116800
7,D8,2200
8,D9,1680000
9,D10,520000


In [ ]:
demand_met = finalresults.groupby(['DemandPoint'])[['load']].sum().reset_index()


In [70]:
result_df['M1_diff']

0     138247.4
1      37635.6
2     119369.6
3         -0.0
4          0.0
5      46116.0
6       5801.4
7          0.0
8          0.0
9      67773.4
10     16570.0
Name: M1_diff, dtype: object